## Seção: Verificar existência dos CSVs
Esta célula verifica se os três arquivos CSV necessários existem em `data/`.
Se algum arquivo estiver faltando, será impresso um erro e a execução será interrompida.

In [3]:
# Bibliotecas
from pathlib import Path
import sys

In [ ]:
data_dir = Path('data')
required = ['all_data.csv', 'id_name.csv', 'steam_app_data.csv']
missing = []
for name in required:
    if not (data_dir / name).exists():
        missing.append(name)

if missing:
    msg = f'Erro: arquivos ausentes em {data_dir}: {missing}'
    print(msg)
    sys.exit(1)

print('Todos os arquivos CSV requeridos estão presentes.')

Todos os arquivos CSV requeridos estão presentes.
